In [4]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, shutil
import pydicom
from pydicom.data import get_testdata_file, get_testdata_files
from pydicom.filereader import read_dicomdir
from skimage.draw import polygon
import SimpleITK as sitk
import sys
import scipy.ndimage as ndi
from skimage import morphology
from scipy import signal

## CompAnaモデルで肺をセグメンテーションしたあと、結節をセグメンテーションする。


In [27]:
def crop_img_and_dilatedmask(img,dilated_array):
    ##dilationしたあと、その他の白い部分を消去する。

    z_up = np.nonzero(dilated_array)[0].max() #zの上端
    z_dn = np.nonzero(dilated_array)[0].min() #zの下端
    y_up = np.nonzero(dilated_array)[1].max() #yの上端
    y_dn = np.nonzero(dilated_array)[1].min() #yの下端
    x_up = np.nonzero(dilated_array)[2].max() #xの上端
    x_dn = np.nonzero(dilated_array)[2].min() #xの下端
    print(z_dn,z_up,x_dn,x_up,y_dn,y_up)

    return img[z_dn:z_up,y_dn:y_up,x_dn:x_up], dilated_array[z_dn:z_up,y_dn:y_up,x_dn:x_up]

image_prediction_dir = '/mnt/hdd/LCSG_impacts/compana_impacts_ps256Tr/'
image_dir = '/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task701_AMOS22/imagesTr/'
new_dir = '/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task802_Impactscrop/imagesTr/'


lung_seg_predict_lists = glob.glob(os.path.join(image_prediction_dir,'*nii.gz'))
Impacts_lists = glob.glob(os.path.join(image_dir,'*.nii.gz'))
pred_paths = glob.glob(image_dir+'/*.nii.gz')

for pred_path in sorted(pred_paths):
    
    default_img_path = image_dir + os.path.basename(pred_path)#.split('.')[0] + '_0000.nii.gz'
    default_img = sitk.ReadImage(default_img_path)

    print(pred_path,default_img.GetMetaData('dim[1]'),default_img.GetMetaData('dim[2]'),default_img.GetMetaData('dim[3]'),default_img.GetMetaData('pixdim[1]'),default_img.GetMetaData('pixdim[2]'),default_img.GetMetaData('pixdim[3]'))



/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task701_AMOS22/imagesTr/amos_0001_0000.nii.gz 768 768 90 0.570312 0.570312 5
/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task701_AMOS22/imagesTr/amos_0004_0000.nii.gz 512 512 78 0.782 0.782 5
/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task701_AMOS22/imagesTr/amos_0005_0000.nii.gz 768 768 80 0.56901 0.56901 5
/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task701_AMOS22/imagesTr/amos_0006_0000.nii.gz 512 512 99 0.835 0.835 5
/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task701_AMOS22/imagesTr/amos_0007_0000.nii.gz 768 768 107 0.518229 0.518229 5
/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task701_AMOS22/imagesTr/amos_0009_0000.nii.gz 512 512 90 0.782 0.782 5
/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task701_AMOS22/imagesTr/amos_0010_0000.nii.gz 512 512 132 0.808 0.808 5
/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/Task701_AMOS22/imagesTr/amos_0011_0000.nii.gz 512 512 92 0.782 0.782 5
/mnt/ssd/nnUNet_raw_data_base/nnUNet_raw_data/

In [16]:
default_img.GetMetaData('dim[0]')

'3'

In [15]:


for k in default_img.GetMetaDataKeys():
    v = default_img.GetMetaData(k)
    print(f'({k}) = = "{v}"')

(ITK_FileNotes) = = "Time=90539.000"
(ITK_original_direction) = = "[UNKNOWN_PRINT_CHARACTERISTICS]
"
(ITK_original_spacing) = = "[UNKNOWN_PRINT_CHARACTERISTICS]
"
(aux_file) = = ""
(bitpix) = = "16"
(cal_max) = = "0"
(cal_min) = = "0"
(datatype) = = "4"
(descrip) = = "Time=90539.000"
(dim[0]) = = "3"
(dim[1]) = = "768"
(dim[2]) = = "768"
(dim[3]) = = "104"
(dim[4]) = = "1"
(dim[5]) = = "1"
(dim[6]) = = "1"
(dim[7]) = = "1"
(dim_info) = = "0"
(intent_code) = = "0"
(intent_name) = = ""
(intent_p1) = = "0"
(intent_p2) = = "0"
(intent_p3) = = "0"
(nifti_type) = = "1"
(pixdim[0]) = = "0"
(pixdim[1]) = = "0.614583"
(pixdim[2]) = = "0.614583"
(pixdim[3]) = = "5"
(pixdim[4]) = = "0"
(pixdim[5]) = = "0"
(pixdim[6]) = = "0"
(pixdim[7]) = = "0"
(qform_code) = = "1"
(qform_code_name) = = "NIFTI_XFORM_SCANNER_ANAT"
(qoffset_x) = = "216"
(qoffset_y) = = "-410.385"
(qoffset_z) = = "72.5"
(quatern_b) = = "0"
(quatern_c) = = "1"
(quatern_d) = = "0"
(scl_inter) = = "-1024"
(scl_slope) = = "1"
(sform_cod

In [6]:
default_array,dilated_array = crop_img_and_dilatedmask(default_array,pred_array)

0 138 0 509 0 350


In [29]:
np.nonzero(dilated_array)[0].max()

137

## dilation https://stackoverflow.com/questions/62412913/how-to-speed-up-dilating-a-3d-region-in-a-boolean-numpy-array

In [69]:
print((pred_array==0).sum())

40735594


In [75]:
from scipy.ndimage import morphology
from scipy import signal

In [2]:
len([1])

1